In [1]:
from imaster_paper_args import *

Start_client: No scheduler file, will start local cluster at  ./temp_skylens/pid2577524/


/hildafs/projects/phy200017p/andypark/.conda/envs/skylite/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8811 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34613 instead
  warnings.warn(
/hildafs/home/andypark/.conda/envs/skylens/lib/python3.9/contextlib.py:126: UserWarning: Creating scratch directories is taking a surprisingly long time. (100.01s) This is often due to running workers on a network file system. Consider specifying a local-directory to point workers to write scratch data to a local disk.
  next(self.gen)


In [2]:
from astropy.io import fits
import os
import glob
import itertools
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pyhalofit
import pandas as pd
import matplotlib.transforms as transforms
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from matplotlib import ticker
from matplotlib import colors
import healpy as hp

import sys
sys.path.insert(0, '../skylens')
sys.path.insert(0, '../hscy1/3_meas_vs_theory')
from wl.ximod import *
import correction
import treecorr
import pyccl
import warnings
warnings.filterwarnings('ignore')

full_sky = 4 * np.pi * (180 / np.pi) ** 2
f_sky = 137. / (full_sky)
fid = 0.82 * np.sqrt((0.233+0.046)/0.3)
ntomo = 4
npair = ntomo * (ntomo + 1) // 2

In [3]:
def plot_scatter(s8_min_f, s8_min_r, labels=None, ret=False):
    fid = 0.82 * np.sqrt(0.279 / 0.3)
    majorLocator = MultipleLocator(0.05)
    majorFormatter = FormatStrFormatter('%g')
    minorLocator = MultipleLocator(0.01)

    fig = plt.figure(figsize=(20, 8))
    ax = fig.add_subplot(121)
    if len(s8_min_f) == 1:
        ax.scatter(s8_min_f[0], s8_min_r[0],
                   marker='x', linewidths=1.1, label=labels[0], c='k')
    else:
        for i, z in enumerate(zip(s8_min_f, s8_min_r)):
            ax.scatter(z[0], z[1],
                       marker='x', linewidths=1.1, label=labels[i], c=f'C{i+1}')
    ax.scatter(fid, fid,
               marker='+', c='r', s=300, linewidths=1.3, label=r'input $S_8$')
    ax.set_ylim(0.7, 0.88)
    ax.set_xlim(0.7, 0.88)
    ax.plot([0, 1], [0, 1], transform=ax.transAxes, c='gray', ls='--')
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_ticks_position('both')
    ax.xaxis.set_major_locator(majorLocator)
    ax.xaxis.set_major_formatter(majorFormatter)
    ax.xaxis.set_minor_locator(minorLocator)
    ax.yaxis.set_major_locator(majorLocator)
    ax.yaxis.set_major_formatter(majorFormatter)
    ax.yaxis.set_minor_locator(minorLocator)
    ax.tick_params(axis='both', which='major', direction='in', 
                   length=16, width=1.1, labelsize=16, pad=10)
    ax.tick_params(axis='both', which='minor', direction='in',
                   length=9, width=1.05)
    ax.set_xlabel(r'$\mathrm{S}_8~$(PS)', fontsize=16)
    ax.set_ylabel(r'$\mathrm{S}_8~$(TPCF$\to$PS)', fontsize=16)
    for axis in ['top', 'bottom', 'left', 'right']:
        ax.spines[axis].set_linewidth(1.3)
    legend = ax.legend(loc='upper left', fontsize=16, frameon=False)
    for handle, text in zip(legend.legendHandles, legend.get_texts()):
        text.set_color(handle.get_facecolor()[0])

    ax2 = fig.add_subplot(122)
    ax2.hist(s8_min_f[0], alpha=0.6, density=True, color='C0', histtype='step', label=r'$\mathrm{S}_8~$(PS)')
    ax2.axvline(np.nanmean(s8_min_f), c='C0', ls='--')
    for i in range(len(s8_min_r)):
        ax2.hist(s8_min_r[i], alpha=0.6, density=True, color=f'C{i+1}', histtype='step', label=labels[i])#label=r'$\mathrm{S}_8~$(TPCF$\to$PS)')
        ax2.axvline(np.nanmean(s8_min_r[i]), c=f'C{i+1}', ls='--')
    ax2.set_xlim(0.7, 0.88)
    ax2.xaxis.set_major_locator(majorLocator)
    ax2.xaxis.set_major_formatter(majorFormatter)
    ax2.tick_params(axis='both', which='major', 
                   length=8, width=1.1, labelsize=16, pad=10)
    ax2.axvline(fid, c='red', ls='--')
    ax2.legend(loc='upper left', fontsize=20)
    ax2.set_xlabel(r'$S_8$', fontsize=20)
    ax2.set_ylabel('N', fontsize=20)
    if ret:
        plt.close(fig)
        return fig, ax, ax2
default_label = r'$\mathrm{S}_8~$(TPCF$\to$PS)'

# Load $C_\ell$ from Halofit

In [4]:
cl_t = fits.getdata('../data/cl.fits')

# Load Coupling Matrices

In [5]:
Mswp = np.zeros((npair, 8000, 8000))
Mscp = fits.getdata('../data/M2p_taper_final.fits')
Mscm = fits.getdata('../data/M2m_taper_final.fits')
for c in range(npair):
    Mswp[c] = fits.getdata(f'../data/Mswp_{c}.fits')

In [6]:
def bin_2d_coupling(
        M=[],
        bin_utils=None,
        wt_b=None,
        wt0=None,
        partial_bin_side=None,
        lm=0,
        lm_step=-1,
        cov=False,
    ):
        ndim = 1
        if cov:
            ndim = 2
        binning_mat = bin_utils["binning_mat"]

        if len(wt0.shape) == 1:
            binning_mat2 = wt0[:, None] * binning_mat * wt_b
        else:
            binning_mat2 = (
                wt0 @ binning_mat @ wt_b
            ) 

        binning_mat = bin_utils["binning_mat_r_dr"]
        if partial_bin_side is None:
            cov_b = binning_mat.T @ M @ binning_mat2
        elif partial_bin_side == 1:
            cov_b = binning_mat.T @ M @ binning_mat2[lm : lm + lm_step, :]
        elif partial_bin_side == 2:
            cov_b = binning_mat[lm : lm + lm_step, :].T @ M @ binning_mat2
        return cov_b

In [7]:
lmin_cl_Bins = 20
lmax_cl_Bins = 3800
Nl_bins = 20
l_bins = np.linspace(lmin_cl_Bins, lmax_cl_Bins, Nl_bins+1)

# For binning survey window coupling matrix
sw_bu = binning()
sw_bu = sw_bu.bin_utils(np.arange(8000), l_bins)

cl_bu = binning()
cl_bu = cl_bu.bin_utils(np.arange(7000), l_bins)

# Load reconstruction from TPCF and PCL

In [38]:
fits.writeto('../data/ell.fits', bc)

In [8]:
bc = cl_bu['bin_center']
w = (bc > 400) & (bc < 1800) # scale cut in ell space

# shape of reconstruction is (number of mocks x number of tomo pairs x number of data points)
cl_pcl = fits.getdata('../data/pcltocl_final_hscy1.fits')[:, :, w]
cl_pcl_mean = np.mean(cl_pcl, axis=0)
cl_pcl_dv = cl_pcl.reshape(cl_pcl.shape[0], -1)
cov_pcl = np.cov(cl_pcl_dv, rowvar=False)
cor_pcl = np.corrcoef(cl_pcl_dv, rowvar=False)

cl_xi = fits.getdata('../data/xitocl_final_hscy1.fits')[:, :, w]
cl_xi_mean = np.mean(cl_xi, axis=0)
cl_xi_dv = cl_xi.reshape(cl_xi.shape[0], -1)
cov_xi = np.cov(cl_xi_dv, rowvar=False)
cor_xi = np.corrcoef(cl_xi_dv, rowvar=False)

N = cl_pcl.shape[0]

# Define useful mappings

In [ ]:
# possible bin pairs (e.g. (0, 0), (2, 3))
bp = list(itertools.combinations_with_replacement(range(ntomo), 2))
print('bp', bp)

# index to bin-pair (e.g. 0 -> (0, 0), 3 -> (0, 3))
did = {i: pair for i, pair in enumerate(bp)}

# bin-pair to index
iid = {value:key for key, value in did.items()}

# index to cov index
ind = np.arange(w.sum() * npair).reshape(npair, w.sum())
ind = {i: ind[i] for i in range(npair)}

In [ ]:
start = {3:0, 2:1, 1:2, 0:3}
fig, axs = plt.subplots(nrows=4, ncols=4, sharey='row', figsize=(20, 20))
for i, row in enumerate(range(3, -1, -1)):
    for j in range(4):
        b = (i, j)
        if j >= start[row]:
            axs[row, j].semilogx(bc[w], cl_pcl_mean[iid[b]] / cl_xi_mean[iid[b]] - 1, '.')
            axs[row, j].set_xscale('log')
            axs[row, j].tick_params(axis='both', labelsize=20)
            axs[row, j].set_xlim(500, 2350)
            axs[row, j].set_ylim(-0.04, 0.04)
            axs[row, j].axhline(-0.01, c='gray', ls='--')
            axs[row, j].axhline(0.01, c='gray', ls='--')
            axs[row, j].axhline(0, c='red', ls='--')
            axs[row, j].axvspan(bc[0], 400, color='gray', alpha=0.3)
            axs[row, j].axvspan(1800, 2350, color='gray', alpha=0.3)
            axs[row, j].text(0.05, 0.8, f'{i+1}x{j+1}', fontfamily='serif', fontsize=15, transform=axs[row, j].transAxes)
            c += 1
        else: 
            axs[row, j].axis('off')
plt.subplots_adjust(wspace=0, hspace=0)
axs[3, 0].set_ylabel(r'$C^{F}_\ell / C^{R}_\ell - 1$', fontsize=22, fontfamily='serif');

# Correlation matrix

In [ ]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(30, 30))
im1 = axs[0].imshow(cor_pcl, origin='lower', cmap='coolwarm', vmax=1., vmin=-1.)
im2 = axs[1].imshow(cor_xi, origin='lower', cmap='coolwarm', vmax=1., vmin=-1.)
plt.colorbar(im1, ax=axs[0], fraction=0.046, pad=0.04)
plt.colorbar(im2, ax=axs[1], fraction=0.046, pad=0.04)
# https://stackoverflow.com/questions/18195758/set-matplotlib-colorbar-size-to-match-graph

# Ratio of covariance matrices

In [ ]:
cmap='coolwarm'
fig, axs = plt.subplots(nrows=1, ncols=1, figsize=(15, 15))
im1 = axs.imshow(cov_pcl / cov_xi - 1, origin='lower', cmap='coolwarm')
plt.colorbar(im1, ax=axs, fraction=0.046, pad=0.04)
# https://stackoverflow.com/questions/18195758/set-matplotlib-colorbar-size-to-match-graph

# Cross correlation

In [ ]:
xticks_labels = [r'$C_{\ell, F}^{0, 0}$',
                 r'$C_{\ell, F}^{0, 1}$',
                 r'$C_{\ell, F}^{0, 2}$',
                 r'$C_{\ell, F}^{0, 3}$',
                 r'$C_{\ell, F}^{1, 1}$',
                 r'$C_{\ell, F}^{1, 2}$',
                 r'$C_{\ell, F}^{1, 3}$',
                 r'$C_{\ell, F}^{2, 2}$',
                 r'$C_{\ell, F}^{2, 3}$',
                 r'$C_{\ell, F}^{3, 3}$',
                 r'$C_{\ell, F}^{0, 0}$',
                 r'$C_{\ell, F}^{0, 1}$',
                 r'$C_{\ell, F}^{0, 2}$',
                 r'$C_{\ell, F}^{0, 3}$',
                 r'$C_{\ell, F}^{1, 1}$',
                 r'$C_{\ell, F}^{1, 2}$',
                 r'$C_{\ell, F}^{1, 3}$',
                 r'$C_{\ell, F}^{2, 2}$',
                 r'$C_{\ell, F}^{2, 3}$',
                 r'$C_{\ell, F}^{3, 3}$',
                ] 
xTticks_labels = [r'$C_{\ell, F}^{0, 0}$',
                 r'$C_{\ell, F}^{0, 1}$',
                 r'$C_{\ell, F}^{0, 2}$',
                 r'$C_{\ell, F}^{0, 3}$',
                 r'$C_{\ell, F}^{1, 1}$',
                 r'$C_{\ell, F}^{1, 2}$',
                 r'$C_{\ell, F}^{1, 3}$',
                 r'$C_{\ell, F}^{2, 2}$',
                 r'$C_{\ell, F}^{2, 3}$',
                 r'$C_{\ell, F}^{3, 3}$',
                 r'$C_{\ell, R}^{0, 0}$',
                 r'$C_{\ell, R}^{0, 1}$',
                 r'$C_{\ell, R}^{0, 2}$',
                 r'$C_{\ell, R}^{0, 3}$',
                 r'$C_{\ell, R}^{1, 1}$',
                 r'$C_{\ell, R}^{1, 2}$',
                 r'$C_{\ell, R}^{1, 3}$',
                 r'$C_{\ell, R}^{2, 2}$',
                 r'$C_{\ell, R}^{2, 3}$',
                 r'$C_{\ell, R}^{3, 3}$',
                ] 
yticks_labels = [r'$C_{\ell, F}^{0, 0}$',
                 r'$C_{\ell, F}^{0, 1}$',
                 r'$C_{\ell, F}^{0, 2}$',
                 r'$C_{\ell, F}^{0, 3}$',
                 r'$C_{\ell, F}^{1, 1}$',
                 r'$C_{\ell, F}^{1, 2}$',
                 r'$C_{\ell, F}^{1, 3}$',
                 r'$C_{\ell, F}^{2, 2}$',
                 r'$C_{\ell, F}^{2, 3}$',
                 r'$C_{\ell, F}^{3, 3}$',
                 r'$C_{\ell, R}^{0, 0}$',
                 r'$C_{\ell, R}^{0, 1}$',
                 r'$C_{\ell, R}^{0, 2}$',
                 r'$C_{\ell, R}^{0, 3}$',
                 r'$C_{\ell, R}^{1, 1}$',
                 r'$C_{\ell, R}^{1, 2}$',
                 r'$C_{\ell, R}^{1, 3}$',
                 r'$C_{\ell, R}^{2, 2}$',
                 r'$C_{\ell, R}^{2, 3}$',
                 r'$C_{\ell, R}^{3, 3}$',
                ] 
yRticks_labels = [r'$C_{\ell, R}^{0, 0}$',
                 r'$C_{\ell, R}^{0, 1}$',
                 r'$C_{\ell, R}^{0, 2}$',
                 r'$C_{\ell, R}^{0, 3}$',
                 r'$C_{\ell, R}^{1, 1}$',
                 r'$C_{\ell, R}^{1, 2}$',
                 r'$C_{\ell, R}^{1, 3}$',
                 r'$C_{\ell, R}^{2, 2}$',
                 r'$C_{\ell, R}^{2, 3}$',
                 r'$C_{\ell, R}^{3, 3}$',
                 r'$C_{\ell, R}^{0, 0}$',
                 r'$C_{\ell, R}^{0, 1}$',
                 r'$C_{\ell, R}^{0, 2}$',
                 r'$C_{\ell, R}^{0, 3}$',
                 r'$C_{\ell, R}^{1, 1}$',
                 r'$C_{\ell, R}^{1, 2}$',
                 r'$C_{\ell, R}^{1, 3}$',
                 r'$C_{\ell, R}^{2, 2}$',
                 r'$C_{\ell, R}^{2, 3}$',
                 r'$C_{\ell, R}^{3, 3}$',
                ] 

In [ ]:
cross_corr = np.corrcoef(cl_pcl_dv, cl_xi_dv, rowvar=False)
fig, ax = plt.subplots(1, 1, figsize=(25, 25))
cross_corr_im = ax.imshow(cross_corr, origin='lower', cmap='coolwarm', vmin=-1., vmax=1.)
axT = ax.secondary_xaxis('top')
axR = ax.secondary_yaxis('right')
plt.colorbar(cross_corr_im, ax=ax, fraction=0.046, pad=0.04)
ax.axvline(x=w.sum()*npair-0.5, c='black', linewidth=5, ls='--')
ax.axhline(y=w.sum()*npair-0.5, c='black', linewidth=5, ls='--')
ax.tick_params(axis='both', pad=3, length=0)
axR.tick_params(axis='y', pad=3, length=0)
axT.tick_params(axis='x', pad=3, length=0)
ax.set_xticks(ticks=np.arange(w.sum()/2, w.sum()*npair*2, w.sum()), labels=xticks_labels, fontsize=20);
ax.set_yticks(ticks=np.arange(w.sum()/2, w.sum()*npair*2, w.sum()), labels=yticks_labels, fontsize=20);
axT.set_xticks(ticks=np.arange(w.sum()/2, w.sum()*npair*2, w.sum()), labels=xTticks_labels, fontsize=20);
axR.set_yticks(ticks=np.arange(w.sum()/2, w.sum()*npair*2, w.sum()), labels=yRticks_labels, fontsize=20);

In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=4, sharey='all', sharex=True, figsize=(25, 25))
for i in range(4):
    for j in range(i, 4):
        b = (i, j)
        axs[i, j].loglog(bc[w], np.diag(cov_pcl)[ind[iid[b]]], label=r'$F_\ell$')
        axs[i, j].loglog(bc[w], np.diag(cov_xi)[ind[iid[b]]], label=r'$\xi$')
        axs[i, j].set_xscale('log')
        axs[i, j].set_xlabel(r'$\ell_\mathrm{max}$', fontsize=22)
        axs[i, j].set_ylabel(r'Diagonal of Cov', fontsize=22)
        axs[i, j].tick_params(axis='both', labelsize=20)
        axs[i, j].set_title(f'{i}, {j}')
        axs[i, j].legend(loc='upper left', fontsize=22)
    for j in range(i):
        axs[i, j].axis('off')

In [ ]:
def get_err(cov):
    return np.sqrt(np.diag(cov))
def corr_mat(cov):
    err=get_err(cov)
    return cov/np.outer(err,err)

def matrix_cut(mat=[],x=[]):
    m=mat[x]
    N=sum(x)
    m2=np.zeros((N,N))
    j=0
    for i in m:
        m2[j]=i[x]
        j=j+1
    return m2

def SN_cum(cov=[],lb=[],cl=[],diag=False,lmin=2,lmax=1e4,use_hartlap=False,nsim=1000):
    sni=np.zeros_like(lb)
    for i in np.arange(len(lb)):
        if lb[i]<lmin or lb[i]>lmax:
            continue
        x=lb<=lb[i]
        x*=lb>lmin
        cov2_cut=matrix_cut(mat=cov,x=x)
        if diag:
            cov2_cut=np.diag(np.diag(cov2_cut))
        cov2_cut_inv=np.linalg.inv(cov2_cut)
        
        cl_i=cl[x]
        SN2=cl_i@cov2_cut_inv@cl_i
        if use_hartlap:
            SN2*=(nsim-2-x.sum())/(nsim-1)
        sni[i]=SN2
    return np.sqrt(sni)

In [ ]:
sn_pcl = np.zeros((npair, w.sum()))
sn_xi = np.zeros((npair, w.sum()))
for c in range(npair):
    sn_pcl[c] = SN_cum(cov=cov_pcl[ind[c], :][:, ind[c]], lb=bc[w], cl=cl_pcl_mean[c])
    sn_xi[c] = SN_cum(cov=cov_xi[ind[c], :][:, ind[c]], lb=bc[w], cl=cl_xi_mean[c])
    
fig, axs = plt.subplots(nrows=4, ncols=4, sharey='all', sharex=True, figsize=(25, 25))
for i in range(4):
    for j in range(i, 4):
        b = (i, j)
        axs[i, j].loglog(bc[w], sn_pcl[iid[b]], label=r'$F_\ell$')
        axs[i, j].loglog(bc[w], sn_xi[iid[b]], label=r'$\xi$')
        axs[i, j].set_xscale('log')
        axs[i, j].set_xlabel(r'$\ell_\mathrm{max}$', fontsize=22)
        axs[i, j].set_ylabel(r'$S/N(\ell_\mathrm{max})$', fontsize=22)
        axs[i, j].tick_params(axis='both', labelsize=20)
        axs[i, j].set_title(f'{i}, {j}')
        axs[i, j].legend(loc='upper left', fontsize=22)
    for j in range(i):
        axs[i, j].axis('off')

In [ ]:
mocks = np.random.randint(0, high=N, size=3)
start = {3:0, 2:1, 1:2, 0:3}
shift = 1.05
for mock in mocks:
    fig, axs = plt.subplots(nrows=4, ncols=4, sharey='row', sharex='col', figsize=(20, 20))
    for i, row in enumerate(range(3, -1, -1)):
        for j in range(4):
            b = (i, j)
            if j >= start[row]:
                std_pcl = np.sqrt(np.diag(cov_pcl[ind[iid[b]], :][:, ind[iid[b]]]))
                std_xi = np.sqrt(np.diag(cov_xi[ind[c], :][:, ind[c]]))
                axs[row, j].semilogx(bc[w], cl_pcl[mock, c] / cl_xi[mock, c] - 1, '.', c='C0', ms=5, label=r'$C_\ell^{F_\ell}$')
                axs[row, j].axhline(0, c='red')
                axs[row, j].axhline(0.01, c='gray', ls='--')
                axs[row, j].axhline(-0.01, c='gray', ls='--')
                axs[row, j].semilogx()
                axs[row, j].tick_params(axis='both', labelsize=20)
                axs[row, j].set_xlim(500, 2350)
                axs[row, j].axvspan(bc[w][0]-100, bc[w][0], color='gray', alpha=0.3)
                axs[row, j].axvspan(bc[w][-1], 2350, color='gray', alpha=0.3)
                axs[row, j].text(0.05, 0.8, f'{i+1}x{j+1}', fontfamily='serif', fontsize=15, transform=axs[row, j].transAxes)
                axs[row, j].tick_params(axis='both', direction='in', which='major', length=15)
                axs[row, j].tick_params(axis='both', direction='in', which='minor', length=7)
                axs[row, j].xaxis.set_major_locator(ticker.MultipleLocator(1000))
                axs[row, j].xaxis.set_minor_locator(ticker.MultipleLocator(200))
                axs[row, j].xaxis.set_minor_formatter('')
                axs[row, j].xaxis.set_major_formatter(ticker.ScalarFormatter(useMathText=True))
                axs[row, j].xaxis.set_ticks_position('both')
                axs[row, j].yaxis.set_ticks_position('both')
                axs[row, j].yaxis.set_minor_locator(ticker.LogLocator(base=10, subs=np.arange(2, 10) * 0.1))
                axs[row, j].yaxis.set_major_formatter(ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
                axs[row, j].yaxis.set_minor_formatter('')
                axs[row, j].set_ylim(-0.1, 0.1)
            else: 
                axs[row, j].axis('off')

    axs[2, 2].legend(bbox_to_anchor=(-1, 2), 
                     loc='upper left', numpoints=1, frameon=False,
                    fontsize=22, markerscale=10)
    plt.subplots_adjust(wspace=0, hspace=0)
    for i in range(4):
        axs[i, start[i]].tick_params(axis='y', which='major', reset=True, labelsize=15)
        axs[i, start[i]].tick_params(axis='both', direction='in', which='major', length=12)
        axs[i, start[i]].tick_params(axis='both', direction='in', which='minor', length=4)
    axs[3, 0].set_xlabel(r'$\ell$', fontsize=22, fontfamily='serif')

In [ ]:
cross_corr = np.corrcoef(cl_xi_dv, cl_pcl_dv, rowvar=False)
one_corr = np.tile(cor_xi, (2, 2))
plt.figure(figsize=(20, 20))
cross_corr_im = plt.imshow(cross_corr - one_corr, origin='lower', cmap='coolwarm', vmin=-0.1, vmax=0.1)
plt.colorbar(cross_corr_im, ax=plt.gca(), fraction=0.046, pad=0.04)

# Load S8 grid and their corresponding $C_\ell, D_\ell, F_\ell, \xi^\pm$

In [ ]:
ngrid = 1000
s8_grid = np.linspace(0.4, 1.2, ngrid)
cells = fits.getdata('../data/cells_grid.fits')
pells = fits.getdata('../data/pells.fits')
fells = fits.getdata('../data/fells.fits')

cells_b = bin_1d(cells[:, :, :7000], cl_bu)
fells_b = bin_1d(fells, cl_bu)

In [ ]:
Mswp_b = np.zeros((npair, Nl_bins, Nl_bins))
Mscp_b = np.zeros_like(Mswp_b)
for c in range(npair):
    Mscp_b[c] = bin_2d_coupling(Mscp, cl_bu, wt_b=1./bin_1d(cl_t[c, :7000], cl_bu), wt0=cl_t[c, :7000])
    Mswp_b[c] = bin_2d_coupling(Mswp[c], sw_bu, wt_b=1./bin_1d(cl_t[c, :8000], sw_bu), wt0=cl_t[c, :8000])

# Reconstruct $C_\ell$ from theory $F_\ell$

In [ ]:
rcells_fb = np.einsum('cij,gcj->gci', np.linalg.pinv(Mscp_b @ Mswp_b), fells_b)

In [ ]:
M2p_final = np.zeros((npair, 8000, 8000))
M2m_final = np.zeros((npair, 8000, 8000))
for c in range(npair):
    M2p_final[c] = fits.getdata(f'../data/M2p_{c}.fits')
    M2m_final[c] = fits.getdata(f'../data/M2m_{c}.fits')
xi_w = fits.getdata('../data/window_xi.fits')

In [ ]:
def taper(l=[], large_l_lower=1000, large_l_upper=1500, low_l_lower=10, low_l_upper=50):
    taper_f = np.zeros_like(l, dtype="float64")
    x = l > large_l_lower
    taper_f[x] = np.cos(
        (l[x] - large_l_lower) / (large_l_upper - large_l_lower) * np.pi / 2.0
    )
    x = np.logical_and(l <= large_l_lower, l >= low_l_upper)
    taper_f[x] = 1
    x = l < low_l_upper
    taper_f[x] = np.cos(
        (l[x] - low_l_upper) / (low_l_upper - low_l_lower) * np.pi / 2.0
    )

    x = np.logical_or(l <= low_l_lower, l >= large_l_upper)
    taper_f[x] = 0
    taper_f = {"taper_f": taper_f, "l": l}
    return taper_f

def bin_2d_inv_WT(
        wig_mat=[],
        wig_norm=None,
        bin_utils_xi=None,
        bin_utils_cl=None,
        wt_b=None,
        wt0=None,
        use_binned_l=False,
        win_xi=None,
    ):

        wig_mat = wig_mat.T * wig_norm
        #bin_win_xi = np.dot(win_xi * bin_utils_xi['r_dr'], bin_utils_xi['binning_mat'])/bin_utils_xi['norm']
        if bin_utils_xi is not None:
            binning_mat_xi = bin_utils_xi["binning_mat"]
            if wt_b is None:
                wt_b = bin_utils_xi["wt_b"]
            if wt0 is None:
                wt0 = bin_utils_xi["wt0"]
            if len(wt0.shape) == 1:
                binning_mat_xi2 = wt0[:, None] * binning_mat_xi / win_xi[:, None] * wt_b
            else:
                binning_mat_xi2 = wt0 @ binning_mat_xi @ wt_b  # FIXME: Test this.

            wm = wig_mat @ binning_mat_xi2

        else:
            wm = wig_mat

        wig_mat_b = wm
        if bin_utils_cl is not None and use_binned_l:
            bin_mat_cl = bin_utils_cl["binning_mat"]

            rdr = bin_utils_cl["r_dr"]
            bin_mat_cl = bin_mat_cl * rdr[:, None] / bin_utils_cl["norm"][None, :]

            wig_mat_b = bin_mat_cl.T @ wm
        return wig_mat_b

cor = treecorr.GGCorrelation(nbins=2400, min_sep=0.25,max_sep=360*2,sep_units='arcmin')
meanr = cor.rnom
corrs=[corr_ll]
corr=corrs[0]
s=s1_s2s[corr]
 
WT_kwargs = {
    'l': np.arange(8000), 'l_cut_weights': None, 'theta': meanr/60 * d2r, 's1_s2': [(0, 0), (2, 2), (2, -2)],
    'wig_d_taper_order_low': None, 'wid_d_taper_order_high': None
}
WT = wigner_transform(**WT_kwargs)
WT.gather_data()
taper_f = taper(meanr, low_l_lower=meanr[0], low_l_upper=10, large_l_lower=320, large_l_upper=360)['taper_f']

# Get theory $\xi^\pm$

In [ ]:
xi_p = np.zeros((npair, 2400))
xi_m = np.zeros((npair, 2400))
l_xi = np.arange(8000)
for i in range(4):
    for j in range(i, 4):
        b = (i, j)
        xi_p[iid[b]] = WT.projected_correlation(l_xi, cl_t[iid[b], :8000], s1_s2=(2, 2))[1]
        xi_m[iid[b]] = WT.projected_correlation(l_xi, cl_t[iid[b], :8000], s1_s2=(2, -2))[1]
xips = fits.getdata('../data/cltoxip.fits')
xims = fits.getdata('../data/cltoxim.fits')

In [ ]:
th = meanr
theta_min = 0.25
theta_max = 480
theta_max = 380
Nt_bins = 200
theta_bins = np.linspace(theta_min, theta_max, Nt_bins+1)
xi_bu = binning()
xi_bu = xi_bu.bin_utils(th, theta_bins)

In [ ]:
Hp = np.zeros((10, Nl_bins, Nt_bins))
Hm = np.zeros_like(Hp)
M2p_final_b = np.zeros((10, Nl_bins, Nl_bins))
for c in range(10):
    Hp[c] = bin_2d_inv_WT(wig_mat=WT.wig_d[(2, 2)], wig_norm=WT.inv_wig_norm*xi_w[c],
                 bin_utils_xi=xi_bu, bin_utils_cl=sw_bu, wt_b=1./bin_1d(xi_p[c], xi_bu), wt0=xi_p[c],
                  use_binned_l=True, win_xi=xi_w[c]
                 )
    Hm[c] = bin_2d_inv_WT(wig_mat=WT.wig_d[(2, -2)], wig_norm=WT.inv_wig_norm*xi_w[c],
                 bin_utils_xi=xi_bu, bin_utils_cl=sw_bu, wt_b=1./bin_1d(xi_m[c], xi_bu), wt0=xi_m[c],
                  use_binned_l=True, win_xi=xi_w[c]
                 )
    M2p_final_b[c] = bin_2d_coupling(M=M2p_final[c], 
                                     bin_utils=sw_bu, wt_b=1./bin_1d(cl_t[c, :8000], sw_bu), wt0=cl_t[c, :8000])

# Reconstruct $C_\ell$ from theory $\xi^\pm$

In [ ]:
xips_b = bin_1d(xips * taper_f * xi_w[None, :, :], xi_bu)
xims_b = bin_1d(xims * taper_f * xi_w[None, :, :], xi_bu)
dells = 0.5 * (np.einsum('clt, nct -> ncl', Hp, xips_b) + np.einsum('clt, nct -> ncl', Hm, xims_b))
rcells_rb = np.einsum('cij, ncj -> nci', np.linalg.pinv(M2p_final_b), dells)

# Run S8 Inference

In [ ]:
runs = N * 1
runs = 100
diff_f = (rcells_fb[:, None, :, w] - cl_pcl[None, :runs, :, :]).reshape(1000, runs, -1)
diff_r = (rcells_rb[:, None, :, w] - cl_xi[None, :runs, :, :]).reshape(1000, runs, -1)
chi2_f = np.einsum('gni, ij, jng -> gn', diff_f, np.linalg.pinv(cov_pcl), diff_f.transpose(2, 1, 0))
chi2_r = np.einsum('gni, ij, jng -> gn', diff_r, np.linalg.pinv(cov_xi), diff_r.transpose(2, 1, 0))
s8_min_f = s8_grid[np.argmin(chi2_f, axis=0)]
s8_min_r = s8_grid[np.argmin(chi2_r, axis=0)]
plot_scatter([s8_min_f], [s8_min_r], labels=[default_label])

# Forward model the fiducial theory

In [ ]:
pcl_t = np.einsum('cij, cj -> ci', Mswp, cl_t[:, :8000])
fcl_t = np.einsum('ij, cj -> ci', Mscp, pcl_t[:, :7000])
cl_tb_pcl = np.einsum('cij,cj->ci', np.linalg.pinv(Mscp_b @ Mswp_b), bin_1d(fcl_t, cl_bu))

In [ ]:
xi_pb = bin_1d(xi_p * taper_f * xi_w, xi_bu)
xi_mb = bin_1d(xi_m * taper_f * xi_w, xi_bu)
dcl_t = 0.5 * (np.einsum('clt, ct -> cl', Hp, xi_pb) + np.einsum('clt, ct -> cl', Hm, xi_mb))
cl_tb_xi = np.einsum('cij, cj -> ci', np.linalg.pinv(M2p_final_b), dcl_t)

# Add shape noise to covariance

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(35, 10))
axs = axs.flatten()
for b in range(10):
    axs[b].semilogx(bc[w], cl_pcl_mean[b] / cl_tb_pcl[b, w] - 1, c='blue', label=r'$F_\ell$')
    axs[b].semilogx(bc[w], cl_xi_mean[b] / cl_tb_xi[b, w] - 1, c='orange', label=r'$\xi$')
    axs[b].axhline(0, c='red', ls='--')
    axs[b].tick_params(axis='both', which='major', labelsize=16)
    i, j = did[b]
    axs[b].text(0.85, 0.90, f'{i+1}x{j+1}', transform=axs[b].transAxes, fontsize=16)
    if b == 0:
        axs[b].legend(fontsize=16, loc='lower left')
    axs[b].set_xlabel(r'$\ell$', fontsize=17)
    axs[b].set_ylim(-0.1, 0.1)
    axs[b].axhline(-0.02, c='gray')
    axs[b].axhline(0.02, c='gray')
axs[0].set_ylabel(r'$\frac{\hat{C}_\ell}{C_\ell} - 1$', fontsize=22);
axs[5].set_ylabel(r'$\frac{\hat{C}_\ell}{C_\ell} - 1$', fontsize=22);

In [ ]:
def _shape_noise_cov_block(cl, i, j, m, n, sn, size):
    cov = np.zeros(size)
    if j == n:
        cov += cl[ind[iid[(i, m)]]] * sn[j]
    if i == m:
        cov += cl[ind[iid[(j, n)]]] * sn[i]
    if i == m and j == n:
        cov += sn[i] * sn[j]
    if j == m:
        cov += cl[ind[iid[(i, n)]]] * sn[j]
    if i == n:
        cov += cl[ind[iid[(j, m)]]] * sn[i]
    if i == n and j == m:
        cov += sn[i] * sn[j]
    return np.diag(cov)

def shape_noise_cov(cl, sn, size, norm):
    cov = np.zeros((10 * size, 10 * size))
    for row in range(10):
        for col in range(row, 10):
            i, j = did[row]
            m, n = did[col]
            cov[row*w.sum():(row+1)*w.sum(), col*w.sum():(col+1)*w.sum()] = _shape_noise_cov_block(cl, i, j, m, n, sn, size) * norm
    return cov

In [ ]:
cl_pcl_mean_dv = cl_pcl_mean.reshape(-1)
cl_xi_mean_dv = cl_xi_mean.reshape(-1)

sigma_e = 0.25
dim = (60 * 180 / np.pi) ** 2 # per rad^2
n_eff = np.array([3.77, 5.07, 4.00, 2.12]) * dim # per arcmin^2

shape_noise =  sigma_e ** 2 / 2. / n_eff
SN_cov_norm = 4 * np.pi / (f_sky * (2 * bc[w] + 1) * np.gradient(bc[w]))

SN_cov_pcl = shape_noise_cov(cl_pcl_mean_dv, shape_noise, w.sum(), SN_cov_norm)
SN_cov_pcl = SN_cov_pcl + SN_cov_pcl.T - np.diag(np.diag(SN_cov_pcl))

SN_cov_xi = shape_noise_cov(cl_xi_mean_dv, shape_noise, w.sum(), SN_cov_norm)
SN_cov_xi = SN_cov_xi + SN_cov_xi.T - np.diag(np.diag(SN_cov_xi))

new_cov_pcl = cov_pcl + SN_cov_pcl
new_cov_xi = cov_xi + SN_cov_xi

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=5, figsize=(35, 10), sharey=True)
axs = axs.flatten()
for b in range(10):
    axs[b].loglog(bc[w], np.diag(new_cov_pcl[ind[b], :][:, ind[b]]), c='blue', label=r'$F_\ell$ with SN')
    axs[b].loglog(bc[w], np.diag(cov_pcl[ind[b], :][:, ind[b]]), ls='--', c='blue', label=r'$F_\ell$ without SN')
    axs[b].loglog(bc[w], np.diag(new_cov_xi[ind[b], :][:, ind[b]]), c='orange', label=r'$\xi$ with SN')
    axs[b].loglog(bc[w], np.diag(cov_xi[ind[b], :][:, ind[b]]), ls='--', c='orange', label=r'$\xi$ without SN')
    axs[b].tick_params(axis='both', which='major', labelsize=16)
    i, j = did[b]
    axs[b].text(0.85, 0.90, f'{i+1}x{j+1}', transform=axs[b].transAxes, fontsize=16)
    if b == 0:
        axs[b].legend(fontsize=16, loc='lower left')
    axs[b].set_xlabel(r'$\ell$', fontsize=17)

In [ ]:
runs = N * 1.
runs = N
diff_f = (rcells_fb[:, None, :, w] - cl_pcl[None, :runs, :, :]).reshape(1000, runs, -1)
diff_r = (rcells_rb[:, None, :, w] - cl_xi[None, :runs, :, :]).reshape(1000, runs, -1)
chi2_f = np.einsum('gni, ij, jng -> gn', diff_f, np.linalg.pinv(new_cov_pcl), diff_f.transpose(2, 1, 0))
chi2_r = np.einsum('gni, ij, jng -> gn', diff_r, np.linalg.pinv(new_cov_xi), diff_r.transpose(2, 1, 0))
s8_min_f = s8_grid[np.argmin(chi2_f, axis=0)]
s8_min_r = s8_grid[np.argmin(chi2_r, axis=0)]
plot_scatter([s8_min_f], [s8_min_r], labels=[default_label], ret=False)